# Image Classification using Pretrained ResNet50

This project demonstrates image classification using a pretrained ResNet50 model (ImageNet weights).

### Features:
- Single Image Prediction
- Top-5 Confidence Visualization
- Grad-CAM Explainability
- Batch Image Prediction
- Feature Extraction (Transfer Learning Ready)

**Author:** Ishank Mishra

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [ ]:
# Load Model
def load_model():
    model = ResNet50(weights='imagenet')
    print('ResNet50 Loaded Successfully')
    return model

model = load_model()

In [ ]:
# Image Preprocessing
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img, img_array

In [ ]:
# Prediction Function
def predict_image(model, img_array):
    preds = model.predict(img_array)
    decoded = decode_predictions(preds, top=5)[0]
    print('\nTop 5 Predictions:\n')
    for i in decoded:
        print(f"{i[1]} : {round(i[2]*100,2)}%")
    return decoded, preds

In [ ]:
# Plot Confidence
def plot_top_predictions(decoded_preds):
    labels = [i[1] for i in decoded_preds]
    scores = [i[2] for i in decoded_preds]
    plt.figure(figsize=(8,5))
    plt.barh(labels, scores)
    plt.xlabel('Confidence')
    plt.title('Top 5 Prediction Confidence')
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
# Grad-CAM
def make_gradcam_heatmap(img_array, model, last_conv_layer_name='conv5_block3_out'):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, np.argmax(predictions[0])]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
# Overlay Grad-CAM
def overlay_heatmap(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224,224))
    heatmap = cv2.resize(heatmap, (224,224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed = cv2.addWeighted(img, 1-alpha, heatmap, alpha, 0)
    plt.imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Grad-CAM Visualization')
    plt.show()

In [ ]:
# Batch Prediction
def batch_predict(folder_path, model):
    print('\nBatch Prediction Results:\n')
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        try:
            _, img_array = preprocess_image(img_path)
            preds = model.predict(img_array)
            decoded = decode_predictions(preds, top=1)[0][0]
            print(f"{file} → {decoded[1]} ({round(decoded[2]*100,2)}%)")
        except:
            continue

In [ ]:
# Feature Extraction
feature_model = ResNet50(weights='imagenet', include_top=False)

def extract_features(img_array):
    features = feature_model.predict(img_array)
    print('Feature shape:', features.shape)
    return features

## Conclusion

This project demonstrates:
- Image Classification using pretrained ResNet50
- Confidence Visualization
- Explainable AI (Grad-CAM)
- Batch Inference
- Feature Extraction for Transfer Learning

Future Improvements:
- Fine-tuning on custom dataset
- Deploy using Streamlit or Flask
- Build Image Similarity Search System